In [3]:
%%time
import cvxpy as cp
import numpy as np
import cyecca.lie as lie
import casadi as ca
import control

A = -np.array(ca.DM(lie.se23.elem(ca.DM([0, 0, 0, 0, 0, 0, 0, 0, 0])).ad()))
B = np.eye(9)
Q = np.eye(9)
R = np.eye(9)
K, _, _ = control.lqr(A, B, Q, R)
K
Ap = A - B@K

A_list = []
d = 0.1
for x0 in [-d, d]:
    for x1 in [-d, d]:
        for x2 in [-d, d]:
            for x3 in [-d, d]:
                for x4 in [-d, d]:
                    for x5 in [-d, d]:
                        for x6 in [-d, d]:
                            for x7 in [-d, d]:
                                for x8 in [-d, d]:
                                    Ai = -np.array(ca.DM(lie.se23.elem(ca.DM([x0, x1, x2, x3, x4, x5, x6, x7, x8])).ad())) - B@K
                                    A_list.append(Ai)

n = 9
P = cp.Variable((n, n), 'P', symmetric=True)
gamma = cp.Variable((1, 1), 'gamma')
constraints = [ P >> np.eye(n) ]

for Ai in A_list:
    constraints += [
        cp.bmat([
            [Ai.T*P + P*Ai, P*B],
            [B.T*P, -gamma*np.eye(n)]
        ]) << 0
    ]

prob = cp.Problem(cp.Minimize(gamma), constraints)
#print('problem\n===========\n', prob)

prob.solve()
#print('\nsolution\n==========\n', prob.solution)

print('\nP\n==========\n', P.value)
print('\ngamma\n==========\n', gamma.value)


P
 [[ 1.00000034e+00  1.54579289e-16  2.29991701e-18 -8.41709953e-17
  -1.44783009e-16 -8.42542394e-17 -1.18434865e-16  2.02598138e-17
   2.73600916e-17]
 [ 1.54579289e-16  1.00000034e+00 -3.95504259e-17 -1.19793607e-16
  -1.63924252e-17  3.46608307e-17 -3.09843443e-17  1.95291219e-17
  -2.90252640e-17]
 [ 2.29991701e-18 -3.95504259e-17  1.00000034e+00 -2.59446098e-18
   6.04082079e-17  3.45197799e-17 -1.26553712e-16 -8.72233802e-17
  -1.67247484e-16]
 [-8.41709953e-17 -1.19793607e-16 -2.59446098e-18  1.00000034e+00
  -1.60207143e-17  1.97469742e-16 -2.76303854e-17 -7.25009072e-17
  -4.92095541e-17]
 [-1.44783009e-16 -1.63924252e-17  6.04082079e-17 -1.60207143e-17
   1.00000034e+00  3.40572546e-17 -7.85237522e-17 -4.28199157e-17
  -5.36227552e-17]
 [-8.42542394e-17  3.46608307e-17  3.45197799e-17  1.97469742e-16
   3.40572546e-17  1.00000034e+00 -1.76677095e-17 -1.50414057e-17
   8.04701840e-17]
 [-1.18434865e-16 -3.09843443e-17 -1.26553712e-16 -2.76303854e-17
  -7.85237522e-17 -1.766